<a href="https://colab.research.google.com/github/jacobeturpin/CRX14/blob/master/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

In [0]:
import gzip
import os
from urllib.request import urlretrieve

import pandas as pd

## Data Loading

In [0]:
def batch_download(first_n=None):

  # URLs for the zip files
  links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
    'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
    'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
    'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
  ]

  first_n = first_n or len(links)

  if first_n > len(links):
    raise('Number of files requested exceeds amount available')

  for idx, link in enumerate(links[:first_n]):
      fn = 'images_%02d.tar.gz' % (idx+1)
      print('downloading', fn, '...')
      urlretrieve(link, fn)  # download the zip file
  
  print("Download complete. Please check the checksums")

In [3]:
batch_download(1)

downloading images_01.tar.gz ...
Download complete. Please check the checksums


In [0]:
def extract_gzip(filename):
  f = gzip.open(filename, 'rb')
  file_content = f.read()
  # TODO: implement save decompressed
  f.close()

In [0]:
for fn in os.listdir():
  if '.tar.gz' in fn:
    extract_gzip(fn)

## Modeling

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

### DenseNet

### EfficientNet